In [1]:
import numpy as np
import os.path

In [2]:
class Trellis():
    def __init__(self,edges):
        self.edges = edges
        self.length = len(edges)
    def query(self,check):
        return [self.edges[i] for i in range(self.length)
                if all(check[j] == self.edges[i][j] for j in check)]
    def get_edge_schema(self):
        return {"initial":  len(self.edges[0]["initial"]),
                "terminal": len(self.edges[0]["terminal"]),
                "input":    len(self.edges[0]["input"]),
                "output":   len(self.edges[0]["output"])}

In [3]:
def gen_outer(rate):
    trel = []
    for si in [[i,j] for i in range(2) for j in range(2)]:
        out = [None,None]
        for inp in range(2):
            out[0] = inp ^ si[0]
            out[1] = inp ^ si[0] ^ si[1]
            trel.append({"initial":tuple(si),
                         "terminal":tuple([inp,si[0]]),
                         "input":tuple([inp]),
                         "output":tuple(out)})
    return trel

In [4]:
def gen_inner(m):
    trel = []
    logm = int(np.log2(m))
    for inp in range(m):
        A = ([int(j) for j in format(inp,'#0'+str(logm + 2) + 'b')[2:]])
        for si in range(2):
            PPM = si
            for i in range(len(A)):
                PPM = 2*PPM*(int(i%logm != 0)) + (PPM & 1 ^ A[i])
                if i%logm == (logm - 1):
                    trel.append(
                        {"initial":tuple([si]),
                         "terminal":tuple([PPM&1]),
                         "input":tuple(A),
                         "output":tuple([int(j == PPM) for j in range(m)])
                        })
                    PPM = PPM & 1
    return trel

In [5]:
def to_string(x):
    return ''.join([str(int(i)) for i in x])
def ungroup(U):
    a = []
    for i in U:
        for j in i:
            a.append(j)
    return a

In [6]:
def encode(U,trellis):
    schema = trellis.get_edge_schema()
    msglen = len(U)
    initial = tuple([0 for i in range(schema["initial"])])
    output = []
    for i in range(int(msglen/schema["input"])):
        edge = trellis.query({"initial":initial,
                              "input":tuple(U[i*schema["input"]:(i+1)*schema["input"]])})
        assert(len(edge) == 1)
        output.append(edge[0]["output"])
        initial = edge[0]["terminal"]
    return ungroup(output)

In [7]:
def channel(C,Ks,Kb):
    Y = [np.random.poisson(Kb) + np.random.poisson(Ks) if C[i] == 1 
         else np.random.poisson(Kb) for i in range(len(C))]
    return Y

In [8]:
def interleave(X):
    pi = interleaver_map(len(X))
    return [X[i] for i in pi]
def interleaver_map(N):
    pi_map = [None for i in range(N)]
    pi_map[0] = 0
    b = 221
    for i in range(1,N):
        pi_map[i] = (pi_map[i-1] + b)%N
        b = (b+420)%N
    return pi_map

In [9]:
def quantize(n,C=8):
    return -127 if (C*n<-127) else 127 if (C*n>127) else int(C*n)
def many_max_star(L):
    val = L[0]
    for i in range(1,len(L)):
        val = max_star(val,L[i])
    return val
def max_star(A,B):
    return max(A,B) + max_star_LUT(abs(A-B))
def max_star_LUT(i):
    a = { 0:6, 1:5,  2:5,  3:4,  4:4,  5:3,  6:3,  7:3,  8:3,  9:2, 10:2, 
         11:2,12:2, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:1, 21:1}
    if i < 22:
        return a[i]
    else:
        return 0

In [10]:
def calc_gamma(Y,LLR,trellis,Ks,Kb):
    m = len(trellis.edges[0]["output"])
    gamma = [{} for i in range(int(len(Y)/m))]
    logm = int(np.log2(m))
    for i in range(int(len(Y)/m)):
        y = Y[i*m:i*m+m]
        llr = LLR[i*logm:i*logm+logm]
        for edge in trellis.edges:
            gamma[i][trellis.edges.index(edge)] = quantize(gamma_one(y,Ks,Kb,edge["output"]) + 
                                     gamma_two(llr,edge["input"]))
    return gamma
def gamma_one(Y,Ks,Kb,output):
    c = output.index(max(output))
    return Y[c]*np.log(1+Ks/Kb)
def gamma_two(LLR,A):
    total = 0
    for i in range(len(A)):
        total += LLR[i]*(-1)**(A[i]+1)
    return total

In [11]:
def calc_sgamma(gam,trellis):
    m = len(trellis.edges[0]["output"])
    sgam = [{} for i in range(len(gam))]
    for i in range(len(gam)):
        for edge in gam[i]:
            val = trellis.edges[edge]
            transition = tuple(val["initial"] + val["terminal"])
            if transition in sgam[i]:
                sgam[i][transition] = max_star(gam[i][edge],sgam[i][transition])
            else:
                sgam[i][transition] = gam[i][edge]
    return sgam

In [12]:
def calc_alpha(sgam,init_alpha,trellis):
    m = len(trellis.edges[0]["output"])
    alph = [{} for i in range(len(sgam))]
    alph[0] = {tuple([i]):init_alpha[i] for i in init_alpha.keys()}
    states = init_alpha.keys()
    for i in range(1,len(sgam)):
        for outstate in states:
            for instate in states:
                if outstate in alph[i]:
                    alph[i][tuple([outstate])] = max_star(alph[i-1][tuple([instate])]+
                                                 sgam[i][tuple([instate]+[outstate])],
                                                 alph[i][tuple([outstate])])
                else:
                    alph[i][tuple([outstate])] = alph[i-1][tuple([instate])]+sgam[i][tuple([instate]+[outstate])]
            alph[i][tuple([outstate])] = quantize(alph[i][tuple([outstate])])
    return alph

In [13]:
def calc_beta(sgam,fin_beta,trellis):
    m = len(trellis.edges[0]["output"])
    beta = [{} for i in range(len(sgam))]
    beta[-1] = {tuple([i]):fin_beta[i] for i in fin_beta.keys()}
    states = fin_beta.keys()
    for i in range(len(sgam)-1,0,-1):
        for instate in states:
            for outstate in states:
                if instate in beta[i-1]:
                    beta[i-1][tuple([instate])] = max_star(beta[i][tuple([outstate])]+
                                                 sgam[i][tuple([instate] + [outstate])],
                                                 beta[i-1][tuple([instate])])
                else:
                    beta[i-1][tuple([instate])] = beta[i][tuple([outstate])] + sgam[i][tuple([instate] + [outstate])]
            beta[i-1][tuple([instate])] = quantize(alph[i][tuple([outstate])])
    return beta

In [14]:
def calc_sigma(alpha,beta,gamma,trellis):
    sigma = [{} for i in range(len(gamma))]
    for i in range(1,len(gam)):
        for edge in gam[i]:
            val = trellis.edges[edge]
            sigma[i][edge] = (alpha[i-1][tuple(val["initial"])] + 
                             gamma[i][edge] + 
                             beta[i][tuple(val["terminal"])])
    return sigma

In [15]:
def calc_A(sigma,trellis):
    LLR_A = [0 for i in range(len(sigma))]
    LLR_A[0] = 127
    for i in range(1,len(sigma)):
        maxes = {i:0}

In [16]:
r = [1,2]
m = 16
Ks = 3
Kb = .001
U = [0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0]
inner = Trellis(gen_inner(16))
outer = Trellis(gen_outer(1/2))
X = encode(U,outer)
A = interleave(X)
C = encode(A,inner)
Y = channel(C,Ks,Kb)

In [17]:
LLRAin = [0 for i in range(len(U)*r[1])]
ab_init = {0:0,1:-127}
gam = calc_gamma(Y,LLRAin,inner,Ks,Kb)
sgam = calc_sgamma(gam,inner)
alph = calc_alpha(sgam,ab_init,inner)
beta = calc_beta(sgam,ab_init,inner)
sigma = calc_sigma(alph,beta,gam,inner)

In [40]:
for i in sigma[1:]:
    for j in range(32):
        print(inner.edges[j]["input"],i[j])
    print("=================")

(0, 0, 0, 0) 127
(0, 0, 0, 0) 0
(0, 0, 0, 1) 254
(0, 0, 0, 1) 0
(0, 0, 1, 0) 127
(0, 0, 1, 0) 0
(0, 0, 1, 1) 127
(0, 0, 1, 1) 0
(0, 1, 0, 0) 127
(0, 1, 0, 0) 0
(0, 1, 0, 1) 127
(0, 1, 0, 1) 0
(0, 1, 1, 0) 127
(0, 1, 1, 0) 0
(0, 1, 1, 1) 127
(0, 1, 1, 1) 0
(1, 0, 0, 0) 127
(1, 0, 0, 0) 0
(1, 0, 0, 1) 127
(1, 0, 0, 1) 127
(1, 0, 1, 0) 127
(1, 0, 1, 0) 0
(1, 0, 1, 1) 127
(1, 0, 1, 1) 0
(1, 1, 0, 0) 127
(1, 1, 0, 0) 0
(1, 1, 0, 1) 127
(1, 1, 0, 1) 0
(1, 1, 1, 0) 127
(1, 1, 1, 0) 0
(1, 1, 1, 1) 127
(1, 1, 1, 1) 0
(0, 0, 0, 0) 0
(0, 0, 0, 0) 127
(0, 0, 0, 1) 64
(0, 0, 0, 1) 127
(0, 0, 1, 0) 0
(0, 0, 1, 0) 127
(0, 0, 1, 1) 0
(0, 0, 1, 1) 127
(0, 1, 0, 0) 0
(0, 1, 0, 0) 127
(0, 1, 0, 1) 0
(0, 1, 0, 1) 127
(0, 1, 1, 0) 0
(0, 1, 1, 0) 127
(0, 1, 1, 1) 0
(0, 1, 1, 1) 127
(1, 0, 0, 0) 0
(1, 0, 0, 0) 127
(1, 0, 0, 1) 0
(1, 0, 0, 1) 191
(1, 0, 1, 0) 0
(1, 0, 1, 0) 127
(1, 0, 1, 1) 0
(1, 0, 1, 1) 127
(1, 1, 0, 0) 0
(1, 1, 0, 0) 127
(1, 1, 0, 1) 0
(1, 1, 0, 1) 127
(1, 1, 1, 0) 0
(1, 1, 1, 0) 127
(1, 1